In [1]:
from IPython.core.display import display, HTML
#makes the display bars longer and spread out 100% across the screen width
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np, pandas as pd
pd.set_option('display.max_rows', 8)

import db_queries as db
import vivarium_helpers.id_helper as idh

from vivarium import Artifact

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/model_validation/model2
ndbs
Tue Aug  3 15:41:04 PDT 2021


# Define file paths

In [3]:
# sim results and artifact paths:
# /ihme/costeffectiveness/results/vivarium_ciff_sam/v2.3_wasting_birth_prevalence/ciff_sam/2021_07_26_17_14_31/count_data
# /ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf

artifact_path = '/ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf'

# Load artifact and see what's in it

In [4]:
art = Artifact(artifact_path, filter_terms=['year_start == 2019', 'age_end <= 5'])
print(art)

Artifact containing the following keys:
metadata
	keyspace
	locations
population
	location
	structure
	age_bins
	demographic_dimensions
	theoretical_minimum_risk_life_expectancy
cause
	all_causes
		cause_specific_mortality_rate
	diarrheal_diseases
		prevalence
		incidence_rate
		remission_rate
		disability_weight
		excess_mortality_rate
		cause_specific_mortality_rate
		restrictions
	measles
		prevalence
		incidence_rate
		disability_weight
		excess_mortality_rate
		cause_specific_mortality_rate
		restrictions
	lower_respiratory_infections
		prevalence
		incidence_rate
		remission_rate
		disability_weight
		excess_mortality_rate
		cause_specific_mortality_rate
		restrictions
	protein_energy_malnutrition
		excess_mortality_rate
		cause_specific_mortality_rate
		restrictions
covariate
	live_births_by_sex
		estimate
sequela
	moderate_acute_malnutrition
		disability_weight
	severe_acute_malnutrition
		disability_weight
risk_factor
	child_wasting
		distribution
		categories
		exposure
		rel

# Load the "live births by sex" covariate estimate

In [5]:
art_live_births = art.load('covariate.live_births_by_sex.estimate')
art_live_births

value
sex    year_start year_end parameter                
Female 2019       2020     lower_value  1.600650e+06
                           mean_value   1.771247e+06
                           upper_value  1.975860e+06
Male   2019       2020     lower_value  1.688388e+06
                           mean_value   1.868336e+06
                           upper_value  2.084165e+06

# Look up the Ethiopia live births covariate in GBD

In [6]:
idh.get_entities()

['age_group',
 'age_group_set',
 'cause',
 'cause_set',
 'cause_set_version',
 'covariate',
 'decomp_step',
 'gbd_round',
 'healthstate',
 'indicator_component',
 'life_table_parameter',
 'location',
 'location_set',
 'location_set_version',
 'measure',
 'metric',
 'modelable_entity',
 'sdg_indicator',
 'sequela',
 'sequela_set',
 'sequela_set_version',
 'sex',
 'split',
 'study_covariate',
 'rei',
 'rei_set',
 'rei_set_version',
 'year']

In [7]:
db.get_ids('covariate')

,covariate_id,covariate_name,covariate_name_short,covariate_description
0,2,Legality of Abortion,abortion_legality,"Legality of abortion, 0 to 100 index (0 = not ..."
1,3,Absolute value of average latitude,abs_latitude,Absolute value of the latitude of the country'...
2,5,Adult HIV death rate,adult_hiv_death_rate,UNAIDS preferred adult HIV death rate
3,7,Antenatal Care (1 visit) Coverage (proportion),ANC1_coverage_prop,Proportion of pregnant women receiving any ant...
...,...,...,...,...
887,2454,"Polio3 lagged five year coverage, COVID-inclus...",lagged_polio3_coverage_prop_covid_inclusive,Five-year rolling average of Polio 3-dose cove...
888,2455,Sugar-sweetened beverages (g) [US R/E],ssb_g_unadj_re,"Sugar-sweetened beverages g/p/d available, una..."
889,2456,Processed meat (g) [US R/E],procmeat_g_unadj_re,"Processed meat g/p/d available, unadjusted [US..."
890,2503,Coverage of Postnatal Care Visit,PNC_coverage_prop,proportion of live births for which the mother...


In [8]:
idh.search_id_table('covariate', 'live births', case=False)

,covariate_id,covariate_name,covariate_name_short,covariate_description
30,60,Live Births (thousands),live_births_thousands,Live Births (thousands)
37,84,Live Births 35+ (proportion),pct_births_in_over35s,Proportion of live births by mothers age 35 an...
38,85,Live Births 40+ (proportion),pct_births_in_over40s,Proportion of live births by mothers age 40 an...
381,1106,Live births (by sex),live_births_by_sex,Live births (by sex)
756,2298,Live births (by child's sex and maternal age),live_births_by_sex_age,Live births (by child's sex and maternal age)


In [9]:
live_births_by_sex_id = idh.list_ids('covariate', 'Live births (by sex)')
live_births_by_sex_id

1106

In [10]:
ethiopia_id = idh.list_ids('location', 'Ethiopia')
ethiopia_id

179

In [11]:
help(db.get_covariate_estimates)

Help on function get_covariate_estimates in module db_queries.api.public:

get_covariate_estimates(covariate_id: int, location_id: Union[int, str, List[int]] = 'all', location_set_id: int = 22, location_set_version_id: int = 0, year_id: Union[int, str, List[int]] = 'all', sex_id: Union[int, str, List[int]] = 'all', age_group_id: Union[int, str, List[int]] = 'all', gbd_round_id: Union[int, NoneType] = None, status: str = 'best', decomp_step: Union[str, NoneType] = None, release_id: Union[int, NoneType] = None, model_version_id: Union[int, NoneType] = None) -> pandas.core.frame.DataFrame
    Returns a set of covariate estimates for a given covariate.
    
    Returns default covariate model for a given GBD round/release if no model_version_id
    is specified. Defaults to no demographic subsetting, but specific demographics can
    be specified.
    
    Args:
        covariate_id: ID of covariate
        location_id: Optionally subset query to only return location IDs specified.
       

In [12]:
gbd_live_births = db.get_covariate_estimates(
    covariate_id = live_births_by_sex_id,
    location_id = ethiopia_id,
    year_id = 2019,
    gbd_round_id = idh.list_ids('gbd_round', '2019'),
    decomp_step = 'step4',
)
gbd_live_births

,model_version_id,covariate_id,covariate_name_short,location_id,location_name,year_id,age_group_id,age_group_name,sex_id,sex,mean_value,lower_value,upper_value
0,32376,1106,live_births_by_sex,179,Ethiopia,2019,22,All Ages,1,Male,1.868336e+06,1.688388e+06,2.084165e+06
1,32376,1106,live_births_by_sex,179,Ethiopia,2019,22,All Ages,2,Female,1.771247e+06,1.600650e+06,1.975860e+06


# Check whether Artifact live births match GBD live births for Ethiopia

## Yes, they match!

### But is there an easier way to check this automatically than doing all these dumb data transformations??!!!

In [13]:
art_births_transformed = art_live_births.unstack('parameter')['value'].xs(2020, level='year_end')
art_births_transformed

,parameter,lower_value,mean_value,upper_value
sex,year_start,,,
Female,2019,1.600650e+06,1.771247e+06,1.975860e+06
Male,2019,1.688388e+06,1.868336e+06,2.084165e+06


In [14]:
gbd_births_transformed = (gbd_live_births
 [['sex', 'year_id', 'lower_value', 'mean_value', 'upper_value']]
 .rename(columns={'year_id': 'year_start'})
 .set_index(['sex', 'year_start'])
#  .rename_axis(columns='parameter') # Oh, good, you can leave out the axis name and .equals() still works.
 .sort_index()
)
gbd_births_transformed

,,lower_value,mean_value,upper_value
sex,year_start,,,
Female,2019,1.600650e+06,1.771247e+06,1.975860e+06
Male,2019,1.688388e+06,1.868336e+06,2.084165e+06


In [15]:
gbd_births_transformed.equals(art_births_transformed)

True

# Check the birth rate against other sources


Here's one source: https://www.statista.com/statistics/976887/crude-birth-rate-in-ethiopia/

> Crude birth rate in Ethiopia 2010-2020
> Published by Aaron O'Neill, Jun 30, 2021
>
> This statistic depicts the crude birth rate in Ethiopia from 2010 to 2020. According to the source, the "crude birth rate indicates the number of live births occurring during the year, per 1,000 population estimated at midyear." In 2020, the crude birth rate in Ethiopia amounted to 31.44 live births per 1,000 inhabitants.
 
Their estimate for 2019 is **31.9 births per 1000 inhabitants** (slightly higher than in 2020).

This number agrees with the estimate from the World Bank: https://data.worldbank.org/indicator/SP.DYN.CBRT.IN?locations=ET

Let's get Ethiopia's population data from GBD to compare our numbers.

In [16]:
ethiopia_pop = db.get_population(
    location_id = ethiopia_id,
#     sex_id = [1,2,3],
    year_id = 2019,
    gbd_round_id = idh.list_ids('gbd_round', '2019'),
    decomp_step = 'step4',
)
ethiopia_pop

,age_group_id,location_id,year_id,sex_id,population,run_id
0,22,179,2019,3,1.075912e+08,192


In [17]:
ethiopia_pop.at[0,'population']

107591163.68341173

In [18]:
crude_birth_rate_per_thousand = art_births_transformed / ethiopia_pop.at[0,'population'] * 1000
crude_birth_rate_per_thousand

,parameter,lower_value,mean_value,upper_value
sex,year_start,,,
Female,2019,14.877147,16.462752,18.364519
Male,2019,15.692624,17.365143,19.371153


Our estimated crude birth rate from the artifact is **33.8 births per 1000 inhabitants**, which is pretty close to the 31.9 above.

In [19]:
# Calculate total birth rate for both sexes
crude_birth_rate_per_thousand.sum()

parameter
lower_value    30.569771
mean_value     33.827896
upper_value    37.735672
dtype: float64

# How does this crude birth rate compare with the birth rate in Vivarium?

Rajan said we were seeing around [340 births per 10,000 simulants](https://ihme.slack.com/archives/C018BLX2JKT/p1627689329022000?thread_ts=1627688406.017600&cid=C018BLX2JKT)

### This is 34 births per 1000 simulants, which matches our estimate of 33.8 births per 1000 people

Or equivalently, 338 births per 10,000 people

In [20]:
# Number of births per 1000 simulants in Vivarium, according to Rajan
340 / 10000 * 1000

34.0

In [21]:
# Expected births per 10,000 people
10*crude_birth_rate_per_thousand.sum()

parameter
lower_value    305.697709
mean_value     338.278955
upper_value    377.356720
dtype: float64

## Except that our simulation is not all people, only children under 5!!

# Let's calculate how many births we expect to see per 10,000 simulants under 5

## Looks like we should expect to see around 2,182 births per 10,000 simulants

This matches what Rajan said we need to maintain the population to account for them aging out: Since the entire population of 0-5 year-olds will be replaced every 5 years, [we need 2000 yearly births per 10,000 simulants to replace those aging out](https://ihme.slack.com/archives/C018BLX2JKT/p1627689329022000?thread_ts=1627688406.017600&cid=C018BLX2JKT).


In [22]:
# Check age group id's to pull population under 5
db.get_ids('age_group')

,age_group_id,age_group_name
0,1,Under 5
1,2,Early Neonatal
2,3,Late Neonatal
3,4,Post Neonatal
...,...,...
427,430,< 19 years
428,431,5 to 64
429,432,6 to 8 months
430,433,9 to 11 months


In [23]:
ethiopia_pop_u5 = db.get_population(
    location_id = ethiopia_id,
#     sex_id = [1,2,3],
    age_group_id = idh.list_ids('age_group', 'Under 5'),
    year_id = 2019,
    gbd_round_id = idh.list_ids('gbd_round', '2019'),
    decomp_step = 'step4',
)
ethiopia_pop_u5

,age_group_id,location_id,year_id,sex_id,population,run_id
0,1,179,2019,3,1.667967e+07,192


In [24]:
crude_birth_rate_per_thousand_u5 = art_births_transformed / ethiopia_pop_u5.at[0,'population'] * 1000
crude_birth_rate_per_thousand_u5

,parameter,lower_value,mean_value,upper_value
sex,year_start,,,
Female,2019,95.964102,106.191951,118.459177
Male,2019,101.224286,112.012765,124.952407


In [25]:
# Calculate expected births per 10,000 simulants
10*crude_birth_rate_per_thousand_u5.sum()

parameter
lower_value    1971.883883
mean_value     2182.047165
upper_value    2434.115835
dtype: float64